In [2]:
import numpy as np #матричные вычисления
import random #рандомы для матрицы весов
import math #модуль, экспонента

# Реализация класса для нейронной сети Кохонена
Классифицирует на 9 цветов. Мерой расстояния использую расстояние Чебышева. Нормировка на 255(при обычной нормировке результат сильно не меняется). Сеть нестабильна из-за начальных рандомных весов. Точность в районе 70-90%.

In [10]:
class SOP():
    def __init__(self, n_iter):
        self.n_iter = n_iter
        #self.D = D
        #self.alpha = alpha
        self.n_class = 9
        self.w = []
        self.colorList = []
    
    #нормировка вектора
    def sumSqrt(self,x): #корень суммы квадратов вектора x
        s = 0
        for i in x:
            s+=i**2
        return s**(1/2)
    def norma(self, x): #нормировка вектора x на 255
        X1 = []
        for i in x:
            x1 = []
            s = self.sumSqrt(i)
            for j in i:
                if(s!=0):
                    x1.append(j/255)
                else:
                    x1.append(j)
            X1.append(x1)
        return X1
    
    def euqlid(self, x, w): #расстояние: Евклидово или Чебышева
        dist = []
        for j in range(self.n_class):
            d = 0
            maxd = 0
            for i in range(len(x)):
                #d+=(x[i]-w[i][j])**2 euqlid
                d = math.fabs(x[i]-w[i][j])
                if(d>maxd):
                    maxd = d
            dist.append(maxd)
        return dist
    
    def reWeight(self, k, alpha, w, x, D): #перевесовка
        ind = []
        #print(w)
        w1 = w.T[k]
        distK = self.euqlid(w1, w)
        #print(w1, distK)
        W = w.copy()

        for i in range(len(distK)):
            if(distK[i]<=D):
                ind.append(i)
        #print(ind)
        for j in ind:
            for i in range(len(x)):
                W[i][j]=w[i][j]+alpha*(x[i]-w[i][j])
        return W
    
    def result(self, Xtest):
        res = []
        for x in Xtest:
            testDist = self.euqlid(x, self.w)
            k = np.argmin(testDist)
            res.append(k)
        return res
    
    def colorName(self, Xtrain):
        basisName = {}
        Ytrain = self.result(Xtrain)
        #print(Ytrain)
        colors = ['black', 'red', 'green','blue', 'yellow', 'cyan', 'magenta','white','brown']
        for i in range(len(Ytrain)):
            basisName[Ytrain[i]] = colors[i]
        return basisName
    
    def findName(self, res, resName):
        color = []
        for i in res:
            color.append(resName[i])
        return color
    
    def fit(self):
        self.w = np.array([[random.uniform(0, 0.4) for t in range(self.n_class)] for i in range(3)])
        
        tenColors = [[0, 0, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], [0, 255, 255], [255, 0, 255], [255, 255, 255],[126,41,31]]
        Xtrain = self.norma(tenColors)
        
        alpha = 0.9
        D = 0.03
        for l in range(self.n_iter):
            for t in range(self.n_class):
                x = Xtrain[t]

                dist = self.euqlid(x, self.w)
                k = np.argmin(dist)

                #print(k)
                self.w = self.reWeight(k, alpha, self.w, x, D)

                alpha -= 0.003
                D -= 0.001
        
        #print(self.result(Xtrain))
        self.colorList = self.colorName(Xtrain)
        
        return self.w
    
    def detect(self, test):
        Xtest = self.norma(test)
        
        #print(self.colorList)
        if(len(self.colorList)==self.n_class):
            codes_Ytest = self.result(Xtest)
            colors_Ytest = self.findName(codes_Ytest, self.colorList)

            return colors_Ytest
        else:
            print('Refit, please')

## Пример

Иницилизация

In [11]:
a = SOP(n_iter = 100)

Обучение

In [21]:
a.fit()

array([[  9.97507171e-01,   9.99756384e-01,   1.03803228e-05,
          3.30533293e-04,   9.97799007e-01,   2.93692151e-04,
          7.61277611e-05,   4.92798689e-01,   9.97046939e-01],
       [  9.37352637e-05,   6.62396405e-05,   4.99013581e-06,
          4.72263822e-04,   9.98164049e-01,   9.97720028e-01,
          9.99710860e-01,   1.60372405e-01,   9.97191463e-01],
       [  9.97543131e-01,   7.78957764e-05,   1.90127475e-05,
          9.98415339e-01,   3.68409462e-04,   9.97817018e-01,
          9.83778328e-05,   1.22151640e-01,   9.97306907e-01]])

In [22]:
test = [[4,1,21],[255,255,255], [205,97,248],[126,41,31],[255,106,0]]

Классификация (если возвращет 'Refit, please!', значит нужно заново обучить на тех же данных(аномальные веса на этапе иницилизации))

In [23]:
a.detect(test)

['black', 'white', 'magenta', 'brown', 'red']

In [24]:
a.detect([[255,105,0],[144,134,0],[200,97,106],[166,175,179],[201,7,159],[0,2,3]])

['red', 'brown', 'brown', 'white', 'magenta', 'black']

### True: red, brown, brown, white, magenta, black